In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Prepare the dataset
def load_dataset(file_path, tokenizer, block_size=128):
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size,
    )
    return dataset

# Load your dataset (e.g., code_snippets.txt)
train_dataset = load_dataset("data/code_snippets.txt", tokenizer)

# Set up data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

# Training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-codegen",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# Train the model
trainer.train()

# Save the model
trainer.save_model("./gpt2-codegen")


In [ ]:
from flask import Flask, request, jsonify
from transformers import GPT2LMHeadModel, GPT2Tokenizer

app = Flask(__name__)

# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained("./gpt2-codegen")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

@app.route('/generate_code', methods=['POST'])
def generate_code():
    input_text = request.json.get('inputText', '')
    inputs = tokenizer.encode(input_text, return_tensors='pt')
    outputs = model.generate(inputs, max_length=150)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return jsonify({'generatedCode': generated_text})

@app.route('/debug_code', methods=['POST'])
def debug_code():
    # Placeholder for debugging functionality
    return jsonify({'debugResult': "Debugging not implemented yet."})

@app.route('/refactor_code', methods=['POST'])
def refactor_code():
    # Placeholder for refactoring functionality
    return jsonify({'refactorResult': "Refactoring not implemented yet."})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True)


In [ ]:
import React, { useState } from 'react';
import axios from 'axios';

function CodeGenerator() {
    const [inputText, setInputText] = useState('');
    const [generatedCode, setGeneratedCode] = useState('');

    const handleGenerate = async () => {
        try {
            const response = await axios.post('/generate_code', { inputText });
            setGeneratedCode(response.data.generatedCode);
        } catch (error) {
            console.error('Error generating code:', error);
        }
    };

    return (
        <div>
            <textarea
                value={inputText}
                onChange={(e) => setInputText(e.target.value)}
                placeholder="Describe the functionality..."
            />
            <button onClick={handleGenerate}>Generate Code</button>
            <pre>{generatedCode}</pre>
        </div>
    );
}

export default CodeGenerator;
